In [ ]:
!wget "https://pjreddie.com/media/files/yolov3.weights"

In [ ]:
from __future__ import division
import time
import torch 
import torch.nn as nn
from torch.autograd import Variable
import numpy as np
import cv2 
from util import *
from darknet import Darknet
from preprocess import prep_image, inp_to_image, letterbox_image
import pandas as pd
import random 
import pickle as pkl
import argparse

In [ ]:
def get_test_input(input_dim, CUDA):
    img = cv2.imread("dog-cycle-car.png")
    img = cv2.resize(img, (input_dim, input_dim)) 
    img_ =  img[:,:,::-1].transpose((2,0,1))
    img_ = img_[np.newaxis,:,:,:]/255.0
    img_ = torch.from_numpy(img_).float()
    img_ = Variable(img_)
    
    if CUDA:
        img_ = img_.cuda()
    
    return img_

def prep_image(img, inp_dim):
    """
    Prepare image for inputting to the neural network. 
    
    Returns a Variable 
    """

    orig_im = img
    dim = orig_im.shape[1], orig_im.shape[0]
    img = (letterbox_image(orig_im, (inp_dim, inp_dim)))
    img_ = img[:,:,::-1].transpose((2,0,1)).copy()
    img_ = torch.from_numpy(img_).float().div(255.0).unsqueeze(0)
    return img_, orig_im, dim

def write(x, img):
    c1 = tuple(x[1:3].int())
    c2 = tuple(x[3:5].int())
    cls = int(x[-1])
    label = "{0}".format(classes[cls])
    color = random.choice(colors)
    cv2.rectangle(img, c1, c2,color, 1)
    t_size = cv2.getTextSize(label, cv2.FONT_HERSHEY_PLAIN, 1 , 1)[0]
    c2 = c1[0] + t_size[0] + 3, c1[1] + t_size[1] + 4
    cv2.rectangle(img, c1, c2,color, -1)
    cv2.putText(img, label, (c1[0], c1[1] + t_size[1] + 4), cv2.FONT_HERSHEY_PLAIN, 1, [225,255,255], 1);
    return img

In [ ]:
class Options:
    confidence = .5
    nms_thresh = .4
    cfgfile = 'cfg/yolov3.cfg'
    weightsfile = './yolov3.weights'
    reso = 416

In [ ]:
args = Options()

In [ ]:
confidence = float(args.confidence)
nms_thesh = float(args.nms_thresh)
start = 0
num_classes = 80
CUDA = torch.cuda.is_available()

In [ ]:
bbox_attrs = 5 + num_classes

In [ ]:
model = Darknet(args.cfgfile)
model.load_weights(args.weightsfile)

In [ ]:
model.net_info["height"] = args.reso

In [ ]:
inp_dim = int(model.net_info["height"])

In [ ]:
if CUDA:
    model.cuda()
model.eval()

In [ ]:
image = cv2.imread("dog-cycle-car.png")

In [ ]:
img, ori_img, dim = prep_image(image, inp_dim)

In [ ]:
im_dim = torch.FloatTensor(dim).repeat(1,2)  

In [ ]:
if CUDA:
    im_dim = im_dim.cuda()
    img = img.cuda()

In [ ]:
with torch.no_grad():   
    output = model(Variable(img), CUDA)
output = write_results(output, confidence, num_classes, nms = True, nms_conf = nms_thesh)

In [ ]:
im_dim = im_dim.repeat(output.size(0), 1)
scaling_factor = torch.min(inp_dim/im_dim,1)[0].view(-1,1)

output[:,[1,3]] -= (inp_dim - scaling_factor*im_dim[:,0].view(-1,1))/2
output[:,[2,4]] -= (inp_dim - scaling_factor*im_dim[:,1].view(-1,1))/2

output[:,1:5] /= scaling_factor

for i in range(output.shape[0]):
    output[i, [1,3]] = torch.clamp(output[i, [1,3]], 0.0, im_dim[i,0])
    output[i, [2,4]] = torch.clamp(output[i, [2,4]], 0.0, im_dim[i,1])

In [ ]:
classes = load_classes('data/coco.names')
colors = pkl.load(open("pallete", "rb"))

g = list(map(lambda x: write(x, ori_img), output))

In [ ]:
from PIL import Image
from IPython.display import display

def show(ary):
    display(Image.fromarray(ary))

In [ ]:
show(ori_img)